How different feature extraction methods affect the effectiveness of classification?

In [ ]:
import os
from pathlib import Path
from math import pi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.applications import VGG16, InceptionV3, MobileNetV2


train_dir = Path('./data/train')
train_filepaths = list(train_dir.glob(r'**/*.jpg'))

test_dir = Path('./data/test')
test_filepaths = list(test_dir.glob(r'**/*.jpg'))

val_dir = Path('./data/validation')
val_filepaths = list(val_dir.glob(r'**/*.jpg'))

aug_dir = Path('./data/augmented/')
aug_filepaths = list(aug_dir.glob(r'**/*.jpg')) + list(aug_dir.glob(r'**/*.jpeg'))

data = train_filepaths + test_filepaths + val_filepaths + aug_filepaths

In [ ]:
def paths_to_dataframe(path):
    labels = []
    for i in range(len(path)):
        labels.append(str(path[i]).split(os.sep)[-2])

    labels = pd.Series(labels, name='Label')
    path = pd.Series(path, name='Path').astype(str)

    df = pd.concat([path, labels], axis=1)

    df = df.sample(frac=1).reset_index(drop = True)

    return df
    
data_df = paths_to_dataframe(data)

In [ ]:
data_df = data_df.sample(frac=0.1, random_state=42)
data_df.shape

In [ ]:
k = 2
kf = KFold(n_splits=k, shuffle=True, random_state=42)

X = np.array(data_df['Path'])
y = np.array(data_df['Label'])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", f1)

    classification_rep = classification_report(y_test, y_pred)
    print("Classification Report:")
    print(classification_rep)

    classes = np.unique(y_test)
    class_accuracy = {}
    for cls in classes:
        indices = np.where(y_test == cls)[0]
        class_accuracy[cls] = accuracy_score(y_test[indices], np.array(y_pred)[indices])

    return accuracy, precision, recall, f1, class_accuracy

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def nested_dichotomy(X_train, y_train, X_test):
    unique_labels = np.unique(y_train)
    num_classes = len(unique_labels)
    classifiers = []
    classified_indices = np.full(len(X_train), False)

    for i in unique_labels:
        y_binary = np.where(y_train == i, 1, 0)
        tree = DecisionTreeClassifier()
        tree.fit(X_train, y_binary)
        classifiers.append((tree, i))
        print(i)

    predictions = []
    for tree, positive_label in classifiers:
        binary_prediction = tree.predict(X_test)
        predictions.append(np.where(binary_prediction == 1, positive_label, None))

    results = ['tomato' for _ in range(len(X_test))]
    for i in range(len(predictions)):
        for j in range(len(X_test)):
            if predictions[i][j] is not None:
                results[j] = unique_labels[i]
        

    return results, predictions, classifiers

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input


def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features.flatten()

In [ ]:

def experiment_loop():

    feature_selection = [VGG16(weights='imagenet', include_top=False, pooling='avg'),
                          InceptionV3(weights='imagenet', include_top=False, pooling='avg'),
                          MobileNetV2(weights='imagenet', include_top=False, pooling='avg')]
            
    scores = [[] for _ in range(k)]

    for fold, (train_index, val_index) in enumerate(kf.split(X)):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        for fs_count, fs in enumerate(feature_selection):
            X_train_fs = np.array([extract_features(img_path, fs) for img_path in X_train])
            X_val_fs = np.array([extract_features(img_path, fs) for img_path in X_val])
            results, pred, models = nested_dichotomy(X_train_fs, y_train, X_val_fs)
            accuracy, precision, recall, f1, class_accuracy = metrics(y_val, results)
            scores[fold].append((accuracy, precision, recall, f1, class_accuracy))

    return scores

scores = experiment_loop()


In [ ]:
#calculate avg accuracy
class_accuracy_mobilenet = [fold[-1][-1] for fold in scores]
class_accuracy_inception = [fold[-1][-1] for fold in scores]
class_accuracy_vgg16 = [fold[-1][-1] for fold in scores]

# sum for each class
sum_class_accuracy_mobilenet = {key: 0 for key in class_accuracy_mobilenet[0].keys()}
sum_class_accuracy_inception = {key: 0 for key in class_accuracy_inception[0].keys()}
sum_class_accuracy_vgg16 = {key: 0 for key in class_accuracy_vgg16[0].keys()}

# sum folds
for fold in class_accuracy_mobilenet:
    for key, value in fold.items():
        if key in sum_class_accuracy_mobilenet:
            sum_class_accuracy_mobilenet[key] += value

for fold in class_accuracy_inception:
    for key, value in fold.items():
        if key in sum_class_accuracy_inception:
            sum_class_accuracy_inception[key] += value

for fold in class_accuracy_vgg16:
    for key, value in fold.items():
        if key in sum_class_accuracy_vgg16:
            sum_class_accuracy_vgg16[key] += value


num_folds = len(scores)

avg_class_accuracy_mobilenet = {key: value / num_folds for key, value in sum_class_accuracy_mobilenet.items()}
avg_class_accuracy_inception = {key: value / num_folds for key, value in sum_class_accuracy_inception.items()}
avg_class_accuracy_vgg16 = {key: value / num_folds for key, value in sum_class_accuracy_vgg16.items()}


In [ ]:
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.bar(avg_class_accuracy_mobilenet.keys(), avg_class_accuracy_mobilenet.values(), color='skyblue')
plt.title('Accuracy for Each Class (MobileNet)')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.subplot(1, 3, 2)
plt.bar(avg_class_accuracy_inception.keys(), avg_class_accuracy_inception.values(), color='skyblue')
plt.title('Accuracy for Each Class (InceptionV3)')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.subplot(1, 3, 3)
plt.bar(avg_class_accuracy_vgg16.keys(), avg_class_accuracy_vgg16.values(), color='skyblue')
plt.title('Accuracy for Each Class (VGG16)')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.bar(avg_class_accuracy_mobilenet.keys(), avg_class_accuracy_mobilenet.values(), color='skyblue')
plt.title('Accuracy for Each Class (MobileNet)')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.subplot(1, 3, 2)
plt.bar(avg_class_accuracy_inception.keys(), avg_class_accuracy_inception.values(), color='skyblue')
plt.title('Accuracy for Each Class (InceptionV3)')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.subplot(1, 3, 3)
plt.bar(avg_class_accuracy_vgg16.keys(), avg_class_accuracy_vgg16.values(), color='skyblue')
plt.title('Accuracy for Each Class (VGG16)')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Retrieve the average value for each model
avg_scores = []
for model_scores in scores:
    avg_accuracy = np.mean([fold[0] for fold in model_scores])
    avg_precision = np.mean([fold[1] for fold in model_scores])
    avg_recall = np.mean([fold[2] for fold in model_scores])
    avg_f1 = np.mean([fold[3] for fold in model_scores])
    avg_scores.append((avg_accuracy, avg_precision, avg_recall, avg_f1))

accuracies, precisions, recalls, f1_scores = zip(*avg_scores)

# Define methods
methods = ['MobileNet', 'InceptionV3', 'VGG16']

# Charts
plt.figure(figsize=(14, 10))

plt.subplot(2, 2, 1)
plt.bar(methods, accuracies, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Average Accuracy')
plt.xlabel('Feature extraction method')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(2, 2, 2)
plt.bar(methods, precisions, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Average Precision')
plt.xlabel('Feature extraction method')
plt.ylabel('Precision')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(2, 2, 3)
plt.bar(methods, recalls, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Average Recall')
plt.xlabel('Feature extraction method')
plt.ylabel('Recall')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(2, 2, 4)
plt.bar(methods, f1_scores, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Average F1-Score')
plt.xlabel('Feature extraction method')
plt.ylabel('F1-Score')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
from itertools import combinations
import numpy as np
from scipy import stats

def compute_corr(accuracies):
    """
    Computes the empirical correlation between successive fold accuracies.
    
    Parameters:
    accuracies (list or np.array): A list or array of accuracy scores from cross-validation.

    Returns:
    float: The estimated correlation between successive fold accuracies.
    """
    m = len(accuracies)
    if m < 2:
        raise ValueError("At least two fold accuracies are required to compute correlation.")
    
    # Calculate the correlation between each successive pair of fold accuracies
    successive_correlations = [
        np.corrcoef(accuracies[i], accuracies[i+1])[0, 1]
        for i in range(m - 1)
    ]
    
    # Compute the mean correlation
    mean_corr = np.mean(successive_correlations)
    
    return mean_corr
    
def corrected_t_test(accuracies_1, accuracies_2, corr=0.1):
    """
    Performs the corrected t-test for cross-validation results.

    Parameters:
    accuracies_1 (list or np.array): A list or array of accuracy scores from cross-validation for technique 1.
    accuracies_2 (list or np.array): A list or array of accuracy scores from cross-validation for technique 2.
    corr (float, optional): The correlation between successive folds. If None, it will be estimated empirically.

    Returns:
    float: The t-statistic value.
    float: The p-value corresponding to the t-statistic.
    """
    m1 = len(accuracies_1)
    m2 = len(accuracies_2)
    mean_accuracy_1 = np.mean(accuracies_1)
    mean_accuracy_2 = np.mean(accuracies_2)
    variance_1 = np.var(accuracies_1, ddof=1)
    variance_2 = np.var(accuracies_2, ddof=1)
  
    
    corrected_variance_1 = variance_1 / (m1 * (1 - corr))
    corrected_variance_2 = variance_2 / (m2 * (1 - corr))
    standard_error = np.sqrt(corrected_variance_1 / m1 + corrected_variance_2 / m2)
    
    t_statistic = (mean_accuracy_1 - mean_accuracy_2) / standard_error
    p_value = stats.t.sf(np.abs(t_statistic), df=min(m1, m2) - 1) * 2  # two-tailed p-value
    
    return t_statistic, p_value



In [ ]:

pairs = combinations([(avg_scores[0], 'MobileNet'), (avg_scores[1], 'InceptionV3'), (avg_scores[2], 'VGG16')], 2)

for pair in pairs:
    (technique_1, name_1), (technique_2, name_2) = pair
    
    t_statistic, p_value = corrected_t_test(technique_1, technique_2)
    
    if t_statistic > 0:
        comparison = f"{name_1} is better than {name_2}"
    elif t_statistic < 0:
        comparison = f"{name_2} is better than {name_1}"
    else:
        comparison = f"There is no significant difference between {name_1} and {name_2}"
    
    significance = "significant" if p_value < 0.05 else "not significant"
    
    print(f'{name_1} vs. {name_2}:')
    print(f'T-statistic: {t_statistic}')
    print(f'P-value: {p_value}')
    print(f'Result: {comparison} (p-value {significance})')


In [ ]:

def create_radar_chart(df):
    categories = ['Accuracy', 'Precision', 'Recall', 'F1']
    N = len(categories)

    # Kąt dla każdej osi w wykresie radarowym
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
    
    # Inicjalizacja wykresu
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    
    # Kolory dla każdego eksperymentu
    colors = ['b', 'r', 'g']
    
    for i, (model_name, experiment) in enumerate(df):
        values = experiment
        values += values[:1]
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=model_name, color=colors[i])
        ax.fill(angles, values, colors[i], alpha=0.1)
    
    # Dodanie etykiet osi
    plt.xticks(angles[:-1], categories)
    
    # Ustawienia osi y
    ax.set_rscale('log')
    ax.set_rlabel_position(30)
    plt.yticks([0.2, 0.4, 0.6, 0.8, 1.0], ["0.2", "0.4", "0.6", "0.8", "1.0"], color="grey", size=7)
    plt.ylim(0, 1)
    
    plt.title('Wykres radarowy dla Accuracy, Precision, Recall, F1', size=15, color='blue', y=1.1)
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    plt.show()

models = [('MobileNet', avg_scores[0]), ('InceptionV3', avg_scores[1]), ('VGG16', avg_scores[2])]
create_radar_chart(models)
